In [1]:
# modules 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer

In [5]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_test ['Transported'] = False
df = pd.concat([df_train, df_test], sort = False)
df.head

<bound method NDFrame.head of      PassengerId HomePlanet CryoSleep     Cabin    Destination   Age    VIP  \
0        0001_01     Europa     False     B/0/P    TRAPPIST-1e  39.0  False   
1        0002_01      Earth     False     F/0/S    TRAPPIST-1e  24.0  False   
2        0003_01     Europa     False     A/0/S    TRAPPIST-1e  58.0   True   
3        0003_02     Europa     False     A/0/S    TRAPPIST-1e  33.0  False   
4        0004_01      Earth     False     F/1/S    TRAPPIST-1e  16.0  False   
...          ...        ...       ...       ...            ...   ...    ...   
4272     9266_02      Earth      True  G/1496/S    TRAPPIST-1e  34.0  False   
4273     9269_01      Earth     False       NaN    TRAPPIST-1e  42.0  False   
4274     9271_01       Mars      True   D/296/P    55 Cancri e   NaN  False   
4275     9273_01     Europa     False   D/297/P            NaN   NaN  False   
4276     9277_01      Earth      True  G/1498/S  PSO J318.5-22  43.0  False   

      RoomService  Fo

In [7]:
df.shape[0] == df_train.shape[0] + df_test.shape[0]

True

In [39]:
df.isna().sum()

Destination     0
Deck            0
HomePlanet      0
Transported     0
Age             0
VIP             0
Num             0
CryoSleep       0
Side            0
PassengerId     0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
dtype: int64

In [11]:
df[['Deck', 'Num', 'Side']] = df ['Cabin'].str.split('/', expand = True)
df = df.drop(columns = ['Cabin'])

In [57]:
df.head()

,Deck,Transported,Age,VIP,Num,CryoSleep,Side,RoomService,FoodCourt,ShoppingMall,...,std_amt_spent,mean_amt_spent,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_U,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_U
0,5,False,39.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.000000,0.0,False,True,False,False,False,False,True,False
1,1,True,24.0,0.0,0.0,0.0,6.0,109.0,9.0,25.0,...,227.807375,147.2,True,False,False,False,False,False,True,False
2,6,False,58.0,1.0,0.0,0.0,6.0,43.0,3576.0,0.0,...,3013.383198,2076.6,False,True,False,False,False,False,True,False
3,6,False,33.0,0.0,0.0,0.0,6.0,0.0,1283.0,371.0,...,1373.410427,1035.2,False,True,False,False,False,False,True,False
4,1,True,16.0,0.0,1.0,0.0,6.0,303.0,70.0,151.0,...,223.988169,218.2,True,False,False,False,False,False,True,False


In [15]:
df['Deck'] = df['Deck'].fillna('U')
df['Deck'].value_counts()

Deck
F    4239
G    3781
E    1323
B    1141
C    1102
D     720
A     354
U     299
T      11
Name: count, dtype: int64

In [19]:
df['Num'] = df['Num'].fillna('-1')
df['Side'] = df['Side'].fillna('U')

In [23]:
df['Deck'] = df['Deck'].map({'G':0,'F':1, 'E':2, 'D':3, 'C':4, 'B':5, 'A':6, 'U':7 , 'T' : 8})
df['Side'] = df['Side'].map({'U':-1 , 'P' : 1, 'S':6})

In [25]:
# Dropping the 'Name' Column
df.drop(['Name'], axis = 1, inplace = True)

In [27]:
impute_lis = ['Age', 'VIP', 'Num', 'CryoSleep', 'Side','PassengerId', 'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']

In [29]:
rest = list(set(df.columns) - set(impute_lis))
df_rest = df[rest]

In [33]:
# Filling the missing values using KNN Algorithm (KNNImputer)
imp = KNNImputer()
df_imputed = imp.fit_transform(df[impute_lis])
df_imputed = pd.DataFrame(df_imputed, columns = impute_lis)
df = pd.concat([df_rest.reset_index(drop = True), df_imputed.reset_index(drop = True)], axis = 1)

In [37]:
df['HomePlanet'] = df['HomePlanet'].fillna('U')
df['Destination'] = df['Destination'].fillna('U')

In [41]:
bill_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
df['amt_spent'] = df[bill_cols].sum(axis = 1)
df['std_amt_spent'] = df[bill_cols].std(axis = 1)
df['mean_amt_spent'] = df[bill_cols].mean(axis = 1)

In [47]:
category_colls = ['HomePlanet', 'Destination']
for col in category_colls :
    df = pd.concat([df, pd.get_dummies(df[col], prefix = col)], axis = 1)

In [51]:
df = df.drop(columns = category_colls)

In [55]:
df.drop(['PassengerId'], axis = 1, inplace = True)

In [59]:
df.corr()['Transported'].sort_values(ascending = False)

Transported                  1.000000
CryoSleep                    0.322537
HomePlanet_Europa            0.131977
Destination_55 Cancri e      0.083625
Deck                         0.077959
Side                         0.072057
FoodCourt                    0.035809
HomePlanet_U                 0.006403
HomePlanet_Mars              0.005643
ShoppingMall                 0.005382
Destination_PSO J318.5-22    0.000760
Destination_U               -0.000554
VIP                         -0.018192
Num                         -0.035240
Age                         -0.050276
Destination_TRAPPIST-1e     -0.072731
std_amt_spent               -0.118857
HomePlanet_Earth            -0.119644
amt_spent                   -0.139205
mean_amt_spent              -0.139205
VRDeck                      -0.142222
Spa                         -0.154374
RoomService                 -0.174605
Name: Transported, dtype: float64

In [61]:
df['3_high_cols'] = df['CryoSleep'] + df['HomePlanet_Europa'] + df['Destination_55 Cancri e']
df['3_low_cols'] = df['mean_amt_spent'] + df['amt_spent'] + df['HomePlanet_Earth']
df.corr()['Transported'].sort_values(ascending = False)

Transported                  1.000000
CryoSleep                    0.322537
3_high_cols                  0.283035
HomePlanet_Europa            0.131977
Destination_55 Cancri e      0.083625
Deck                         0.077959
Side                         0.072057
FoodCourt                    0.035809
HomePlanet_U                 0.006403
HomePlanet_Mars              0.005643
ShoppingMall                 0.005382
Destination_PSO J318.5-22    0.000760
Destination_U               -0.000554
VIP                         -0.018192
Num                         -0.035240
Age                         -0.050276
Destination_TRAPPIST-1e     -0.072731
std_amt_spent               -0.118857
HomePlanet_Earth            -0.119644
amt_spent                   -0.139205
mean_amt_spent              -0.139205
3_low_cols                  -0.139229
VRDeck                      -0.142222
Spa                         -0.154374
RoomService                 -0.174605
Name: Transported, dtype: float64

In [73]:
# Modle Creation
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [82]:
# Splitting back the data set into train and test
df_train, df_test = df[:df_train.shape[0]], df[df_train.shape[0]:]
df_test = df_test.drop(columns = 'Transported')
df_train.shape, df_test.shape

((8693, 25), (4277, 24))

In [84]:
x = df_train.drop(columns = 'Transported')
y = df_train['Transported']

#Making 80 % of the data to train the model and the other 20 % to test the models accuracy 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

#Creating Five Models to see which is most accurate
model_1 = LogisticRegression()
model_2 = DecisionTreeClassifier()
model_3 = RandomForestClassifier()
model_4 = XGBClassifier()
model_5 = LGBMClassifier()

In [88]:
#Testing Model 1
model_1.fit(x_train, y_train)
pred = model_1.predict(x_test)
accuracy_score(y_test, pred)

D:\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7734330074755607

In [104]:
#Testing Model 2
model_2.fit(x_train, y_train)
pred = model_2.predict(x_test)
accuracy_score(y_test, pred)

0.7326049453709028

In [92]:
#Testing Model 3
model_3.fit(x_train, y_train)
pred = model_3.predict(x_test)
accuracy_score(y_test, pred)

0.7901092581943646

In [106]:
#Testing Model 4
model_4.fit(x_train, y_train)  
pred = model_4.predict(x_test)
accuracy_score(y_test, pred)

0.79700977573318

In [108]:
#Testing Model 5
model_5.fit(x_train, y_train)
pred = model_5.predict(x_test)
accuracy_score(y_test, pred)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3500, number of negative: 3454
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000508 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2707
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503307 -> initscore=0.013230
[LightGBM] [Info] Start training from score 0.013230


0.7998849913743531

In [112]:
df_dummy = pd.read_csv('test.csv')
pred = model_5.predict(df_test)

final = pd.DataFrame()
final['PassengerId'] = df_dummy['PassengerId']
final['Transported'] = pred

final.to_csv('submission.csv', index = False)